In [1]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from tqdm import tqdm

In [2]:
def pagination(n):
    driver = webdriver.Safari()
    li_url=[]
    for i in range(1,n+1):
        url1='https://stackoverflow.com/jobs?pg='+str(i)
        driver.get(url1)
        obj=driver.find_elements_by_class_name('js-result ')
        dfs=pd.DataFrame()
        for o in obj:
            title=o.find_elements_by_tag_name('h2')
            for t in title:
                url= t.find_element_by_tag_name("a").get_attribute("href")
                li_url.append(url)
    driver.close()
    return li_url 

In [3]:
def webpage_scrap(url1,n):
    driver = webdriver.Safari()
    obj_dict={}
    driver.get(url1)
    try:
        salery=driver.find_element_by_class_name('fc-green-400').text
    except:
        salery="not defined"
    try:
        job_name=driver.find_element_by_class_name('fc-black-900').text
    except:
        job_name="not defined"
    try:
        com_name=driver.find_element_by_class_name('fc-black-800').text
    except:
        com_name="not defined"
    try:
        com_orign=driver.find_element_by_class_name('fc-black-500').text
    except:
        com_name="not defined"
    try:
        loc=driver.find_element_by_class_name('fc-yellow-500').text
    except:
        loc="not given"
    obj=driver.find_elements_by_class_name('mb32')
    about=obj[0:2]
    tag= []
    title=[]
    for o in about:
        j=o.find_elements_by_class_name('mb8')
        for i in j:
            try:
                t=i.find_element_by_tag_name('span').text
                tag.append(t)
                t2=i.find_element_by_class_name('fw-bold').text
                title.append(t2)
            except:
                tag.append("not given")
                title.append("not given")
    dictionary = dict(zip(tag, title))
    # print(df)
    obj.pop(0)
    obj.pop(0)
    techno= obj[0]
    obj.pop(0)
    try:
        tech=techno.find_elements_by_class_name('s-tag')
    except:
        tech="not"
    desc=obj[0]
    obj.pop(0)
    d=desc.find_element_by_tag_name('div').text
    apply=obj[0]
    try:
        app_link=apply.find_element_by_tag_name("a").get_attribute("href")
    except:
        app_link="not given"
    obj_dict[n]={
        "Salary" :salery,
        "Job_title":job_name,
        "Company_name":com_name,
        "Location":loc,
        "Technology/Framework": ",".join([_.text for _ in tech]),
        "Description":d,
        "Apply_link":app_link,
        "url":url1,  
    }
    df = pd.DataFrame(obj_dict).T
    df = df.replace(r'\n',' ', regex=True)
    dfs = pd.DataFrame.from_records([dictionary])
    dfs= dfs.reset_index()
    result = pd.concat([df,dfs], axis=1, join='inner')
    result.drop(['index'],axis=1,inplace=True)
    driver.close()
    return result

In [4]:
driver = webdriver.Safari()
url1='https://stackoverflow.com/jobs'
driver.get(url1)
j = driver.find_element_by_class_name('s-pagination').find_elements_by_tag_name("a")
page_no=int(j[-2].text)
driver.close()
li_url = pagination(3)

In [5]:
dfs=pd.DataFrame()
ddfs=pd.DataFrame()
n=0
for i in li_url:
    df=webpage_scrap(i,n)
    dfs=dfs.append(df)

In [6]:
dfs

,Apply_link,Company_name,Description,Job_title,Location,Salary,Technology/Framework,url,Job type:,Experience level:,Role:,Industry:,Company size:,Company type:,Preferred Timezone:,Employer's note:,Visa Sponsorship:,Relocation Assistance:,Office Location:
0,not given,Staff Engineer (f/m/d),...,Senior Software Engineer (f/m/d) - Backend,not given,not defined,,https://stackoverflow.com/jobs/550537/senior-s...,Full-time,Senior,Backend Developer,Logistics and Supply Chain,201-500 people,VC Funded,NaN,NaN,NaN,NaN,NaN
0,https://crazygames.recruitee.com/o/remote-full...,Lead Mobile developer,"CrazyGames, a fast-growing browser games plat...",Full-stack Typescript Software Engineer,Remote,,"typescript,reactjs,node.js,next.js,graphql",https://stackoverflow.com/jobs/296971/full-sta...,Full-time,"Mid-Level, Senior, Lead",Full Stack Developer,"Computer Games, Platforms, Software Developmen...",11-50 people,Private,(GMT+01:00) Brussels +/- 6 hours,NaN,NaN,NaN,NaN
0,https://jibble-group.breezy.hr/p/61c48ba159e30...,Jibble,About Jibble Group We’re a scale-up in the Wo...,Full Stack Developer,Remote,$48k - 60k,".net,c#,postgresql,vue.js,api",https://stackoverflow.com/jobs/557756/full-sta...,Full-time,Mid-Level,"Backend Developer, Frontend Developer, Full St...",NaN,NaN,NaN,(GMT+04:00) Yerevan +/- 4 hours,NaN,NaN,NaN,NaN
0,not given,Data Engineer at sustainable mobility start-u...,...,VP Engineering at mobility start-up (m/f/x) - ...,Remote,€100k - 125k | Equity,,https://stackoverflow.com/jobs/445056/vp-engin...,Full-time,"Lead, Manager",NaN,"Agile Software Development, Automotive, Mobility",51-200 people,VC Funded,NaN,NaN,NaN,NaN,NaN
0,https://www.codapayments.com/,Technical Lead (Vue.js),...,"Senior/Front End Engineer, APAC",Remote,not defined,,https://stackoverflow.com/jobs/557316/senior-f...,Full-time,"Mid-Level, Senior","Backend Developer, Frontend Developer","Financial Technology, Mobile Payments",201-500 people,VC Funded,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,not given,(Senior) Software Engineer (Ruby) - Logistics...,...,(Senior) Software Engineer (Golang) - Quick Co...,not given,not defined,,https://stackoverflow.com/jobs/552571/senior-s...,Full-time,"Mid-Level, Senior",DevOps,"eCommerce, Food & Beverage, Logistics & Distri...",10k+ people,Public,NaN,NaN,NaN,NaN,NaN
0,not given,Lead Mobile developer,...,Full-Stack Engineer,Remote,€60k - 80k | Equity,,https://stackoverflow.com/jobs/521144/full-sta...,Full-time,"Mid-Level, Senior","Backend Developer, Full Stack Developer",Financial Technology,11-50 people,Private,NaN,NaN,NaN,NaN,NaN
0,https://stackoverflow.com/jobs/apply/558870?ra...,Test Engineer/Cypress developer for a virtual...,Job Description We are a 100% virtual team th...,Xamarin Lead Developer for a virtual team,Remote,DKK 300k - 540k,"c#,scrum,xamarin.forms,xamarin,azure",https://stackoverflow.com/jobs/558870/xamarin-...,Full-time,"Senior, Lead",NaN,"Social Advertising, Social Media",11-50 people,Private,(GMT-02:00) Sao Paulo,NaN,NaN,NaN,NaN
0,https://personio.jobs.personio.de/job/409204?_...,Senior Backend Engineer - Recruiting (m/f/d) ...,THE ROLE At Personio we are on the amazing jo...,Backend Engineer - HR Management,On-site and limited remote,Equity,"amazon-web-services,kotlin,java,go,microservices",https://stackoverflow.com/jobs/558673/backend-...,Full-time,Mid-Level,Backend Developer,"Cloud-Based Solutions, HR Services, Software D...",1k-5k people,VC Funded,NaN,up to 50% remote work,Yes,Yes,NaN


In [7]:
dfs.to_pickle("stackOverflow_jobs.pkl")